# Setup your environnement

## Requirements

```sh
python3 --version
pip3 --version
virtualenv --version
````

## Create a virtualenv and install Tensorflow
```sh 
virtualenv --system-site-packages -p python3 ./venv
source ./venv/bin/activate
pip install --upgrade pip
pip3 install --upgrade tflite_runtime-1.14.0-cp37-cp37m-linux_armv7l.whl
```

## Check you setup is correct
```sh
python -c "import tensorflow as tf;print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

## Only Tensorflow Lite interpreter

python -c "import tflite_runtime.interpreter as tflite"
```

In [ ]:
# import tensorflow as tf
# from tensorflow import lite as tflite

import tflite_runtime.interpreter as tflite
import numpy as np
import pandas as pd
import distro
import matplotlib.pyplot as plt

print('Running Tensorflow on {} {}'.format(distro.name(), distro.version()), distro.codename())

# Load your data

Note: In practice, this should be pre-processed sensors data

In [ ]:
test_dataset = pd.read_pickle('./datasets/test_lg_dataset.pickle')
test_weights = pd.read_pickle('./datasets/test_lg_weights.pickle')

# Load your tflite model

## Feed TFLite Interpreter with your model

In [ ]:
tflite_model = './models/lg_weight.tflite'
interpreter = tflite.Interpreter(model_path=tflite_model)

## or aternatively if you convert the model in this notebook
#interpreter = tf.lite.Interpreter(model_content=tflite_model)

## Allocate Memory

In [ ]:
# you must allocate tensors first
interpreter.allocate_tensors()
# obtaining the input-output shapes and types
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)

## Iterate through the test data and run Tensorflow Lite inference

In [ ]:
%%time
test_tflite_prediction = []
for index,row in test_dataset.iterrows():
    # reshape according to the model input shape
    input_tensor = np.array(row.values, 'f').reshape(input_details[0]['shape'])
    # tensor index can be gotten from the 'index' field in get_input_details
    interpreter.set_tensor(tensor_index=input_details[0]['index'], value=input_tensor)
    # run inference
    interpreter.invoke()
    # get the results
    inference = interpreter.get_tensor(output_details[0]['index'])
    test_tflite_prediction.append(inference.flatten())

## Plot results and compare with True values

In [ ]:
test_tflite_prediction = np.array(test_tflite_prediction).flatten()
plt.rcParams['figure.figsize'] = (8, 8)

def plot_predictions(true, inferences):
    plt.scatter(true, inferences, marker='x')
    plt.xlabel('True Values [Weight]')
    plt.ylabel('Inferences [Weight]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    plt.plot([-0, 250], [-0, 250], color='orange')

    plt.show()

plot_predictions(test_weights, test_tflite_prediction)